
# Example 2: Machine-learning benzene energies.
This example demonstrates:

1. How to import packages

2. How to print hello world


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

## Section 1: print hello



In [ ]:
x = 1
y = 2
print("Hello, world!")

In [ ]:
plt.plot(np.exp(np.linspace(0, 5)))